In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_type_code = df['Type_Code'].tolist()
list_part_key = df['Part_Key'].tolist()

print('总数量：' + str(len(list_part_key)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for type_code, part_key in zip(list_type_code, list_part_key):
    work.put_nowait((type_code, part_key))

def crawler():
    global df
    
    while not work.empty():
        type_code, part_key = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                if part_key == '':
                        df_temp = pd.DataFrame([{'status': 'ok',
                                                 'Part_Key': part_key,
                                                 'Vehicle': ''}])

                        status = 'ok'

                        break
                
                data = 'func=getbuyersguide&payload=' + quote('{"partData":{"listing_data_essential":{"parttype":"') + type_code + quote('","partkey":"') + part_key + quote('"}}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/x-www-form-urlencoded; charset=UTF-8'
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['buyersguidepieces']['body']

                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_vehicle = html.xpath('//div[contains(@class, "buyersguide-nested")]/descendant::table[contains(@class, "nobmp")]/tr')

                vehicle = '\n'.join([' '.join(_.xpath('./td/text()')) for _ in list_vehicle])

                # = = = = = = = = = = = = = = =
           
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Key': part_key,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Key': part_key}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_key +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：380

1514426  <->  [ok] - 剩余数量：370
12769041  <->  [ok] - 剩余数量：369
13953693  <->  [ok] - 剩余数量：368
1514692  <->  [ok] - 剩余数量：367
1513520  <->  [ok] - 剩余数量：366
1513956  <->  [ok] - 剩余数量：365
12769037  <->  [ok] - 剩余数量：364
1515630  <->  [ok] - 剩余数量：363
1514150  <->  [ok] - 剩余数量：362
1515016  <->  [ok] - 剩余数量：361
2415314  <->  [ok] - 剩余数量：360
1515624  <->  [ok] - 剩余数量：359
1515200  <->  [ok] - 剩余数量：358
1515444  <->  [ok] - 剩余数量：357
1514520  <->  [ok] - 剩余数量：356
1515188  <->  [ok] - 剩余数量：355
1515006  <->  [ok] - 剩余数量：354
1515502  <->  [ok] - 剩余数量：353
1515678  <->  [ok] - 剩余数量：352
1514786  <->  [ok] - 剩余数量：351
1513814  <->  [ok] - 剩余数量：350
1514970  <->  [ok] - 剩余数量：349
1515600  <->  [ok] - 剩余数量：348
1513410  <->  [ok] - 剩余数量：347
1513536  <->  [ok] - 剩余数量：346
1514384  <->  [ok] - 剩余数量：345
1515322  <->  [ok] - 剩余数量：344
1513292  <->  [ok] - 剩余数量：343
1514238  <->  [ok] - 剩余数量：342
1514266  <->  [ok] - 剩余数量：341
1514402  <->  [ok] - 剩余数量：340
1514954  <->  [ok] - 剩余数量：339
1515748  <->  [ok] - 剩余数量：33

8606148  <->  [ok] - 剩余数量：97
9484408  <->  [ok] - 剩余数量：96
9484428  <->  [ok] - 剩余数量：95
9484436  <->  [ok] - 剩余数量：94
13953697  <->  [ok] - 剩余数量：93
8767760  <->  [ok] - 剩余数量：92
13953701  <->  [ok] - 剩余数量：91
10206200  <->  [ok] - 剩余数量：90
10085632  <->  [ok] - 剩余数量：89
9484440  <->  [ok] - 剩余数量：88
9484448  <->  [ok] - 剩余数量：87
10206204  <->  [ok] - 剩余数量：86
9484460  <->  [ok] - 剩余数量：85
10085628  <->  [ok] - 剩余数量：84
10206208  <->  [ok] - 剩余数量：83
13953705  <->  [ok] - 剩余数量：82
9484468  <->  [ok] - 剩余数量：81
9484484  <->  [ok] - 剩余数量：80
9484456  <->  [ok] - 剩余数量：79
9484488  <->  [ok] - 剩余数量：78
10085636  <->  [ok] - 剩余数量：77
9484472  <->  [ok] - 剩余数量：76
9484464  <->  [ok] - 剩余数量：75
9484476  <->  [ok] - 剩余数量：74
9484496  <->  [ok] - 剩余数量：73
9484508  <->  [ok] - 剩余数量：72
9484504  <->  [ok] - 剩余数量：71
10085624  <->  [ok] - 剩余数量：70
10206212  <->  [ok] - 剩余数量：69
9484524  <->  [ok] - 剩余数量：68
9484516  <->  [ok] - 剩余数量：67
9719688  <->  [ok] - 剩余数量：66
9484536  <->  [ok] - 剩余数量：65
9484528  <->  [ok] - 剩余数量：64
948